<a href="https://colab.research.google.com/github/Yunsoonsang/MySchoolProject/blob/master/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__, static_folder="static", static_url_path="/static")

cap = cv2.VideoCapture(0)
out = None # cv2.VideoWriter object

def generate_frames():
  while True:
    # frame read
    success, frame = cap.read()
    if not success:
      break

    # save video data
    if out is not None:
      out.write(frame)

    # frame encoding byte
    ret, buffer = cv2.imencode('.jpg', frame)
    frame = buffer.tobytes()

    # transport frame to react
    yield (b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
  return render_template("index.html")

@app.route('/video_feed')
def video_feed():
  # transport video stream to react
  return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/start_capture')
def start_capture():
  global out
  out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'))
  return "Capture started successfully"

@app.route('/stop_capture', methods=['POST'])
def stop_capture():
  global out
  if out is not None:
    out.release()
    out = None
    return "Capture stopped successfully"
  else:
    return "No capture in progress"

@app.route('/upload', methods=['POST'])
def upload():
    # Get image data from react-webcam
    image = request.get_json().get('image')

    # AI code upload
    result = process_ai(image)

    return "Image received successfully"

def process_ai(image):
    # insert ai code
    # ex: result = yolov8_model.process_image(image)
    return "AI processing result"

if __name__ == "__main__":
  app.run(host="0.0.0.0", port=5000, debug=True)